#**Kubernetes homework**

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.base import clone
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
 Remove old zip if exists
if os.path.exists("data.zip"):
    os.remove("data.zip")

# Download new data.zip
!wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip -O data.zip

# Remove existing data/ folder
if os.path.exists("data"):
    shutil.rmtree("data")

# Unzip the new dataset
!unzip -q data.zip